In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyter_progressbar import ProgressBar
import os
from dnrepresentations.parsing import ForumPost, ForumThread, Author

## Now lets scrape all the files with my new classes

In [ ]:
def write_thread_to_database(thread: ForumThread):
    raise NotImplementedError()

In [ ]:
def yield_path_of_html_files(root_path="data/digitalnippon.de"):
    html_folders = ["anime-manga-japan", "digital-nippon", "digital-nippon-archiv", "medientitel", "rpg-forest", "smalltalk"]
    walkers = (os.walk(os.path.join(root_path, directory)) for directory in html_folders)
    for walker in walkers:
        for root, _, files in walker:
            for file in files:
                # Filter non-HTML files
                basename, ext = os.path.splitext(file)
                if ext not in [".htm", ".html"] or basename.endswith("POLL"):
                    continue
                full_path = os.path.join(root, file)
                yield full_path

In [ ]:
nr_files = sum(1 for _ in yield_path_of_html_files())
print(nr_files)

In [ ]:
threads = []
for path in ProgressBar(yield_path_of_html_files(), size=nr_files):
    thread = ForumThread(path)
    try:
        thread.parse()
    except AttributeError as e:
        print(path, e)
        continue
    except Exception:
        print(path)
        raise
    threads.append(thread)

In [ ]:
member_walker = os.walk("data/digitalnippon.de/mitglieder")
member_root, _, member_files = next(member_walker)
authors = []
for file in member_files:
    full_path = os.path.join(member_root, file)
    author = Author(full_path)
    try:
        author.parse()
    except Exception:
        print(full_path)
        raise
    authors.append(author)